# Simulación de una falla en una Máquina Síncrona en el Marco dq

## Objetivo
Simular una falla de 100 ms en los terminales de una máquina síncrona, con \( V_s = 0 \). La simulación incluirá la dinámica del estator y también proporcionará una comparación de resultados sin considerar la dinámica del estator.

## Parámetros

- **Parámetros de la Máquina:**
  - \( M_f = 50 \times 10^{-3} \) H
  - \( L_s = 8 \times 10^{-3} \) H
  - \( M_s = 2 \times 10^{-3} \) H
  - \( L_f = 500 \times 10^{-3} \) H
  - \( L_d = 300 \times 10^{-3} \) H
  - \( L_q = 100 \times 10^{-3} \) H
  - \( M_d = 30 \times 10^{-3} \) H
  - \( M_q = 20 \times 10^{-3} \) H
  - \( L_{fd} = 130 \times 10^{-3} \) H
  - \( R_s = 0 \) Ω
  - \( R_f = 0.3 \) Ω
  - \( R_d = 0.3 \) Ω
  - \( R_q = 0.3 \) Ω
  - \( V_f = 130 \) V
  - \( L_m = 1 \times 10^{-3} \) H
  - \( R_{Load} = 0.5 \) Ω

## Suposiciones

- Existe un motor primario que conduce a la frecuencia según la siguiente función:
  \[
  \theta(t) = 2 \pi \cdot 60 \cdot t
  \]
- Durante la falla, la resistencia de carga \( R_{Load} \) se reduce a cero, y posteriormente se recupera a su valor original después de la duración de la falla.

## Resultados Esperados

1. **Corrientes del Estator:**
   - Graficar las corrientes del estator en el marco dq.
   - Graficar las corrientes del estator en el marco abc.

2. **Potencia de Salida:**
   - Graficar la potencia de salida durante la falla y el período de recuperación.

## Resultados de la Simulación
- Generar gráficos para visualizar:
  - El comportamiento transitorio de las corrientes del estator en los marcos dq y abc.
  - La potencia de salida antes, durante y después de la falla.
  



In [ ]:
import numpy as np
import plotly.graph_objects as go

# Definir parámetros de simulación
t_total = 1.0  # Tiempo total de simulación en segundos
dt = 0.001  # Paso de tiempo
time = np.arange(0, t_total, dt)

# Parámetros de la máquina
Mf = 50E-3
LS = 8E-3
Ms = 2E-3
LF = 500E-3
LD = 300E-3
LQ = 100E-3
MD = 30E-3
MQ = 20E-3
LFD = 130E-3
Rs = 0
Rf = 0.3
RD = 0.3
RQ = 0.3
Vf = 130
Lm = 1E-3
RLoad = 0.5

# Inicialización de variables de estado
i_d = 0.0
i_q = 0.0
i_a = 0.0
i_b = 0.0
i_c = 0.0
p_out = []  # Salida de potencia
currents_dq = []  # Almacenar corrientes dq
currents_abc = []  # Almacenar corrientes abc

# Parámetros de simulación de fallas
fault_duration = 0.1  # Duración de la falla en segundos (100 ms)
fault_start = 0.1  # Inicio de la falla en 0.1 s
fault_end = fault_start + fault_duration  # Fin de la falla

# Bucle (ciclo for) de simulación
for t in time:
    # Determinar si la falla está activa
    if fault_start <= t < fault_end:
        RLoad = 0.0  # La resistencia de carga cae a cero durante la falla
        V_s = 0.0  # La tensión terminal es cero
    else:
        RLoad = 0.5  # La resistencia de carga vuelve a su valor normal

    # Velocidad sincrónica y theta
    theta = 2 * np.pi * 60 * t

    # Voltajes en el eje dq (suponiendo V_s = 0 durante la falla)
    V_d = Vf * np.cos(theta)  # Componente dq
    V_q = Vf * np.sin(theta)

    # Ecuaciones del estator (sin dinámicas del estator)
    di_d_dt = (V_d - Rs * i_d) / LD
    di_q_dt = (V_q - RQ * i_q) / LQ

    # Actualizar corrientes
    i_d += di_d_dt * dt
    i_q += di_q_dt * dt

    # Convertir corrientes dq a corrientes abc
    i_a = i_d
    i_b = (i_d - np.sqrt(3) * i_q) / 2
    i_c = (i_d + np.sqrt(3) * i_q) / 2

    # Calcular la potencia de salida
    p_out.append(i_d * V_d + i_q * V_q)  # Salida de potencia en el marco dq

    # Almacenar valores de corriente
    currents_dq.append((i_d, i_q))
    currents_abc.append((i_a, i_b, i_c))

# Convertir a arreglos de numpy para facilitar la graficación
currents_dq = np.array(currents_dq)
currents_abc = np.array(currents_abc)
p_out = np.array(p_out)

# Graficar los resultados usando Plotly
# Graficar corrientes en el marco dq
fig = go.Figure()

# Corrientes en el marco dq
fig.add_trace(go.Scatter(x=time, y=currents_dq[:, 0], mode='lines', name='i_d (Marco DQ)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=time, y=currents_dq[:, 1], mode='lines', name='i_q (Marco DQ)', line=dict(color='orange')))
fig.update_layout(title='Corrientes del Estator en Marco DQ',
                  xaxis_title='Tiempo (s)',
                  yaxis_title='Corriente (A)',
                  legend=dict(x=0, y=1))

# Graficar corrientes en el marco abc
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=time, y=currents_abc[:, 0], mode='lines', name='i_a (Marco ABC)', line=dict(color='blue')))
fig2.add_trace(go.Scatter(x=time, y=currents_abc[:, 1], mode='lines', name='i_b (Marco ABC)', line=dict(color='orange')))
fig2.add_trace(go.Scatter(x=time, y=currents_abc[:, 2], mode='lines', name='i_c (Marco ABC)', line=dict(color='green')))
fig2.update_layout(title='Corrientes del Estator en Marco ABC',
                   xaxis_title='Tiempo (s)',
                   yaxis_title='Corriente (A)',
                   legend=dict(x=0, y=1))

# Graficar potencia de salida
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=time, y=p_out, mode='lines', name='Potencia de Salida', line=dict(color='purple')))
fig3.update_layout(title='Potencia de Salida',
                   xaxis_title='Tiempo (s)',
                   yaxis_title='Potencia (W)',
                   legend=dict(x=0, y=1))

# Mostrar gráficas
fig.show()
fig2.show()
fig3.show()



## Análisis del Código

### A. Parámetros de Simulación:
* Se definieron los parámetros de la máquina síncrona, incluidos inductancias, resistencias y la carga, que son críticos para modelar el comportamiento del sistema.
* Se estableció el tiempo total de simulación en 1 segundo con un paso de tiempo de 1 ms, lo que permite una resolución temporal adecuada para capturar dinámicas rápidas, como la falla transitoria.

### B. Condiciones de Falla:
* La simulación incluye un evento de falla que ocurre entre 0.1 y 0.2 segundos, durante el cual la resistencia de carga \( R_{Load} \) se reduce a cero y la tensión terminal \( V_s \) se establece en cero.
* Esta condición es típica en simulaciones de fallas para observar cómo el sistema responde a condiciones anormales.

### C. Cálculo de Corrientes:
* Se implementan ecuaciones diferenciales para calcular las corrientes en los ejes dq.
* La implementación simplifica el modelo al no incluir dinámicas del estator, enfocándose en el comportamiento de la máquina y la respuesta a la perturbación.
* Se realiza la conversión de las corrientes dq a corrientes abc, lo que permite una comparación directa entre los marcos de referencia.

### D. Cálculo de Potencia:
* La potencia de salida se calcula a partir de las corrientes en el marco dq y la tensión, lo cual es fundamental para entender el rendimiento del generador durante la falla.

### E. Visualización:
* Se utilizan gráficos de Plotly para representar las corrientes en los marcos dq y abc, así como la potencia de salida.
* Esta representación permite visualizar claramente el comportamiento del sistema antes, durante y después de la falla.


## Resultados Gráficos

### A. Corrientes en el Marco DQ:
* La gráfica de corrientes en el marco dq muestra cómo la corriente \( i_d \) y \( i_q \) responden a la falla.
* Se espera que durante el período de la falla (0.1 - 0.2 s), las corrientes experimenten una caída abrupta, indicando que la máquina no está contribuyendo efectivamente a la carga.
* Después de la falla, las corrientes deberían recuperarse gradualmente a sus niveles normales a medida que se restablece la tensión y la carga.

### B. Corrientes en el Marco ABC:
* La gráfica correspondiente al marco abc refleja la conversión de las corrientes dq.
* Las corrientes \( i_a \), \( i_b \) e \( i_c \) mostrarán patrones similares a los de \( i_d \) e \( i_q \), pero pueden tener una fase diferente debido a la conversión.

### C. Potencia de Salida:
* La gráfica de potencia de salida indicará el impacto de la falla.
* Durante la falla, se espera que la potencia de salida caiga a cero, lo que representa la incapacidad de la máquina para entregar energía a la carga.
* Posteriormente, al restablecerse la carga y la tensión, la potencia de salida debería recuperarse gradualmente.
